In [19]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# LOAD DATASET

In [20]:
test_unknown_data_path = './data/3_rf_test_unknown_data_v1.csv'
test_unknown_data = pd.read_csv(test_unknown_data_path)

test_unknown_label_path = './data/3_rf_test_unknown_label_v1.csv'
test_unknown_label = pd.read_csv(test_unknown_label_path)
test_unknown_data_path2 = './data/3_rf_test_unknown_data_v1.csv'
test_unknown_data2 = pd.read_csv(test_unknown_data_path)

test_unknown_label_path2 = './data/3_rf_test_unknown_label_v1.csv'
test_unknown_label2 = pd.read_csv(test_unknown_label_path)

In [21]:
# Display column names
print("Column names:")
print(test_unknown_data.columns)

# Display dataset shape
print("\nDataset shape:")
print(test_unknown_data.shape)


Column names:
Index(['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate',
       'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight'],
      dtype='object')

Dataset shape:
(5686, 46)


In [22]:
train_data = pd.concat([test_unknown_data,test_unknown_data2], axis=0)
train_label = pd.concat([test_unknown_label, test_unknown_label2], axis=0)

# USING ELBOW METHOD TO FIND OPTIMUM K VALUE

In [23]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Assuming 'test_unknown_data' is your dataset
data = train_data

# Step 4: Data Preprocessing
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# Step 5: Choose the Number of Clusters (K)
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

# Calculate the percentage of variance explained
percentage_var_explained = np.diff(wcss) / np.sum(wcss[:-1]) * 100

# Plotting the elbow graph with the percentage of variance explained
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

plt.subplot(1, 2, 2)
plt.plot(range(2, 11), percentage_var_explained, marker='o')
plt.title('Percentage of Variance Explained')
plt.xlabel('Number of clusters')
plt.ylabel('Percentage of Variance Explained')
plt.show()


AttributeError: module 'numpy' has no attribute 'min'

<Figure size 1000x500 with 2 Axes>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Drop the 'Cluster' column if it exists
data_for_scaling = data.drop('Cluster', axis=1, errors='ignore')

# Step 4: Data Preprocessing
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_scaling)

# Manually set the optimal number of clusters based on the elbow method
your_optimal_k = 3

# Fit K-Means model to the training data
kmeans = KMeans(n_clusters=your_optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=2)
kmeans.fit(scaled_data)

# Assuming your test data has the same features as the training data
scaled_test_data = scaler.transform(data_for_scaling)

# Predict the clusters for the test data
test_cluster_labels = kmeans.predict(scaled_test_data)

# Silhouette Score for clustering evaluation
silhouette_avg = silhouette_score(scaled_test_data, test_cluster_labels)
print(f"Silhouette Score: {silhouette_avg}")

# Plotting predicted clusters
plt.scatter(range(len(test_cluster_labels)), test_cluster_labels, cmap='viridis', marker='o')
plt.xlabel('Data Points')
plt.ylabel('Predicted Clusters')
plt.title('Predicted Clusters')
plt.show()


Silhouette Score: 0.20363773661424941


C:\Users\aravi\AppData\Local\Temp\ipykernel_33768\2335598788.py:33: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(range(len(test_cluster_labels)), test_cluster_labels, cmap='viridis', marker='o')


AttributeError: module 'numpy' has no attribute 'matrix'

<Figure size 640x480 with 1 Axes>

# PERFORM KMEANS AND PLOT PCA

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'test_unknown_data' is your dataset
# and 'test_unknown_label' is your label
X = train_data
y = train_label['label']  # Assuming the label column is named 'label'

# Apply label encoding to the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.4, random_state=42)

# Specify the number of clusters
number_of_clusters = 11  # Adjust this based on your data

# Apply k-means clustering on the training set
kmeans = KMeans(n_clusters=number_of_clusters, random_state=42)
train_predicted_labels = kmeans.fit_predict(X_train)

# Apply the clustering model to the test set
test_predicted_labels = kmeans.predict(X_test)

# Visualize clusters using PCA for dimensionality reduction
pca = PCA(n_components=2)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

# Scatter plot for training set with real labels
plt.figure(figsize=(10, 6))
for cluster in range(number_of_clusters):
    cluster_mask = (train_predicted_labels == cluster)
    plt.scatter(X_train_reduced[cluster_mask, 0], X_train_reduced[cluster_mask, 1], label=f'Cluster {cluster}')

plt.title('K-Means Clustering - Training Set with Real Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

# Scatter plot for test set with real labels
plt.figure(figsize=(10, 6))
for cluster in range(number_of_clusters):
    cluster_mask = (test_predicted_labels == cluster)
    plt.scatter(X_test_reduced[cluster_mask, 0], X_test_reduced[cluster_mask, 1], label=f'Cluster {cluster}')

plt.title('K-Means Clustering - Test Set with Real Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()


c:\Users\aravi\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


AttributeError: module 'numpy' has no attribute 'matrix'

<Figure size 1000x600 with 1 Axes>

AttributeError: module 'numpy' has no attribute 'matrix'

<Figure size 1000x600 with 1 Axes>

# PERFORM K MEANS AND PLOT KMEANS RESULTS

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd

# Assuming 'test_unknown_data' is your dataset
# Assuming 'test_unknown_label' is your label
X = train_data
y_true = train_label['label']

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA for dimensionality reduction to 2 components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Apply k-means clustering on the reduced dataset
number_of_clusters = 11
kmeans = KMeans(n_clusters=number_of_clusters, random_state=42)
predicted_labels = kmeans.fit_predict(X_scaled)

# Create a DataFrame to store the association between clusters and ground truth labels
cluster_label_association = pd.DataFrame({'Cluster': predicted_labels, 'Ground Truth Label': y_true})

# Plot the association using a scatter plot
plt.figure(figsize=(10, 6))

# Iterate over unique ground truth labels
for label in y_true.unique():
    indices = cluster_label_association[cluster_label_association['Ground Truth Label'] == label].index
    plt.scatter(X_pca[indices, 0], X_pca[indices, 1], label=f'Ground Truth - {label}')

# Add legend
plt.legend(title='Legend', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('K-Means Clustering - Scatter Plot with Ground Truth Label Association (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

plt.show()


c:\Users\aravi\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


AttributeError: module 'numpy' has no attribute 'matrix'

<Figure size 1000x600 with 1 Axes>

# FIND FP BY CONSIDERING BENIGN AS POSITIVE LABEL

In [ ]:
# Assuming 'test_unknown_label' is your label
y_true = train_label['label']

# Check unique values in true labels
unique_labels = y_true.unique()
print("Unique Labels:", unique_labels)

from sklearn.metrics import confusion_matrix

# Assuming 'test_unknown_label' is your label
y_true = train_label['label']

# Define the positive class
positive_class = 'BenignTraffic'

# Create binary labels for the positive class
y_binary = (y_true == positive_class).astype(int)

# Evaluate K-means performance using confusion matrix
conf_matrix = confusion_matrix(y_binary, predicted_labels)

# Calculate False Positive Rate for the positive class
false_positive_rate = conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0])

print(f"False Positive Rate for class '{positive_class}': {false_positive_rate}")



Unique Labels: ['BenignTraffic' 'DNS_Spoofing' 'Recon' 'MITM-ArpSpoofing'
 'Backdoor_Malware' 'DictionaryBruteForce' 'CommandInjection' 'DoS' 'XSS'
 'SqlInjection' 'BrowserHijacking' 'VulnerabilityScan']
False Positive Rate for class 'BenignTraffic': 0.782608695652174


# FIND FPR BY CONSIDERING BENIGN AS POSITIVE LABEL

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Assuming 'test_unknown_label' is your true labels
y_true = train_label['label']

# Define the positive class
positive_class = 'BenignTraffic'

# Create binary labels for the positive class
y_binary = (y_true == positive_class).astype(int)

# Evaluate K-means performance using confusion matrix
conf_matrix = confusion_matrix(y_binary, predicted_labels)

# Extract unique class labels (excluding the positive class)
unique_labels = pd.Series(y_true.unique()).loc[lambda x: x != positive_class].tolist()

# Calculate False Positives and True Negatives for the positive class against every other class
false_positives = {}
true_negatives = {}
for i, label in enumerate(unique_labels, start=0):
    false_positives[label] = conf_matrix[i, 0]
    true_negatives[label] = conf_matrix[0, i + 1:].sum() + conf_matrix[i + 1:, i + 1:].sum()

# Calculate False Positive Rate for the positive class against every other class
fpr = {}
for label in unique_labels:
    denominator = false_positives[label] + true_negatives[label]
    # Handle the case where both False Positives and True Negatives are zero
    fpr[label] = false_positives[label] / denominator if denominator != 0 else 0

print(f"False Positive Rate for class '{positive_class}' against every other class:")
for label, fpr_value in fpr.items():
    print(f"Class '{label}': {fpr_value}")


False Positive Rate for class 'BenignTraffic' against every other class:
Class 'DNS_Spoofing': 0.007527286413248024
Class 'Recon': 0.46464646464646464
Class 'MITM-ArpSpoofing': 0.0
Class 'Backdoor_Malware': 0.0
Class 'DictionaryBruteForce': 0.0
Class 'CommandInjection': 0.0
Class 'DoS': 0.0
Class 'XSS': 0.0
Class 'SqlInjection': 0.0
Class 'BrowserHijacking': 0.0
Class 'VulnerabilityScan': 0


In [ ]:
cluster_label_association

,Cluster,Ground Truth Label
0,6,BenignTraffic
1,3,BenignTraffic
2,3,BenignTraffic
3,6,BenignTraffic
4,4,BenignTraffic
...,...,...
5723,4,BenignTraffic
5724,1,Recon
5725,1,BenignTraffic
5726,3,BenignTraffic


In [ ]:
cluster_gtl_counts = cluster_label_association.groupby('Cluster')['Ground Truth Label'].value_counts().unstack(fill_value=0)

In [ ]:
print(cluster_gtl_counts)

Ground Truth Label  Backdoor_Malware  BenignTraffic  BrowserHijacking  \
Cluster                                                                 
0                                  0            414                 0   
1                                  0            620                 1   
2                                  0            124                 0   
3                                  1           1102                 0   
4                                  0            646                 1   
5                                  0              2                 0   
6                                  0            916                 0   
7                                  1             46                 0   
8                                  2           1026                 5   
9                                  0            171                 0   
10                                 0              0                 0   

Ground Truth Label  CommandInjection  DNS_Spoofing

In [ ]:
cluster_b_ratio = (cluster_gtl_counts['BenignTraffic'] / cluster_gtl_counts.sum(axis=1)).fillna(0)

print(cluster_b_ratio)

Cluster
0     0.911894
1     0.811518
2     0.953846
3     0.991007
4     0.853369
5     0.666667
6     0.986006
7     0.851852
8     0.780228
9     0.818182
10    0.000000
dtype: float64
